In [15]:
#imports and setup
import numpy as np
import pandas as pd

pd.set_option('precision', 2) #number precision for pandas
pd.set_option('display.max_rows', 12)
pd.set_option('display.float_format', '{:20,.2f}'.format) #get rid of scientific notation

In [16]:
#load data
auto = pd.read_csv('./Auto.csv', na_values=['?'])
auto.dropna(axis=0, inplace=True)

auto.cylinders = auto.cylinders.astype('category')
auto.name = auto.name.astype('category')

auto.reset_index(inplace=True)

auto['horsepower_2'] = np.power(auto.horsepower, 2)
auto['horsepower_3'] = np.power(auto.horsepower, 3)
auto['horsepower_4'] = np.power(auto.horsepower, 4)
auto['horsepower_5'] = np.power(auto.horsepower, 5)

#Polynomial Features using sklearn
from sklearn.preprocessing import PolynomialFeatures
pol = PolynomialFeatures(degree=5, interaction_only=False, include_bias=False)
polf = pol.fit_transform(auto.loc[:, 'horsepower'].values.reshape(-1, 1))

In [17]:
from sklearn.model_selection import train_test_split
x, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']], auto.mpg
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#ols model with intercept
lm1 = LinearRegression(fit_intercept=True)
lm2 = LinearRegression(fit_intercept=True)
lm3 = LinearRegression(fit_intercept=True)

lm1_fit = lm1.fit(x_train.loc[:, 'horsepower'].values.reshape(-1, 1), y_train)
lm2_fit = lm2.fit(x_train.loc[:, ['horsepower', 'horsepower_2']], y_train)
lm3_fit = lm3.fit(x_train.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']], y_train)

lm1_predict = lm1_fit.predict(x_test.loc[:, 'horsepower'].values.reshape(-1, 1))
lm2_predict = lm2_fit.predict(x_test.loc[:, ['horsepower', 'horsepower_2']])
lm3_predict = lm3_fit.predict(x_test.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3']])

print('lm1 MSE:', mean_squared_error(y_test, lm1_predict))
print('lm2 MSE:', mean_squared_error(y_test, lm2_predict))
print('lm3 MSE:', mean_squared_error(y_test, lm3_predict))

lm1 MSE: 25.573878189684407
lm2 MSE: 22.21802005003286
lm3 MSE: 22.667675435534456


In [22]:
from sklearn.model_selection import LeaveOneOut

x, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3', 'horsepower_4', 'horsepower_5']], auto.mpg
loocv = LeaveOneOut()
loocv.get_n_splits(x)

392

In [26]:
loocv_mse = []
lm = LinearRegression(fit_intercept=True)

for train_index, test_index in loocv.split(x):
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    lm1_fit = lm.fit(x_train.loc[:, 'horsepower'].values.reshape(-1, 1), y_train)
    lm1_predict = lm1_fit.predict(x_test.loc[:, 'horsepower'].values.reshape(-1, 1))
    
    loocv_mse.append(mean_squared_error(y_test, lm1_predict))
    
np.array(loocv_mse).mean()

24.231513517929226

In [28]:
#using sklearn cross_validation_score
from sklearn.model_selection import cross_val_score

lm = LinearRegression(fit_intercept=True)
cval = cross_val_score(lm,
                       auto.loc[:, 'horsepower'].values.reshape(-1, 1),
                       auto.mpg,
                       cv=len(auto), #k=n k-Fold -> LOOCV
                       n_jobs = -1,
                       scoring = 'neg_mean_squared_error')
-cval.mean()

24.23151351792923

In [34]:
#Loop for 5 degree polynomial linear regressions with LOOCV

loocv_poly = {}
for i in range(1, 6):
    loocv_mse = []
    
    for train_index, test_index in loocv.split(x):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if i == 1:
            X_TRAIN = x_train.iloc[:,0:1].values.reshape(-1, 1)
            X_TEST = x_test.iloc[:,0:1].values.reshape(-1, 1)
        else:
            X_TRAIN = x_train.iloc[:,0:i]
            X_TEST = x_test.iloc[:,0:i]
            
        loocv_mse.append(
            mean_squared_error(
                y_test,
                LinearRegression(fit_intercept=True)
                .fit(
                    X_TRAIN,
                    y_train
                )
                .predict(
                    X_TEST
                )
            )
        )
        
        loocv_poly['lm' + str(i) + '_MSE'] = np.array(loocv_mse).mean()

In [35]:
loocv_poly

{'lm1_MSE': 24.231513517929226,
 'lm2_MSE': 19.24821312448967,
 'lm3_MSE': 19.33498406402958,
 'lm4_MSE': 19.4244303102179,
 'lm5_MSE': 19.033212804000605}

In [36]:
from sklearn.model_selection import KFold

x, y = auto.loc[:, ['horsepower', 'horsepower_2', 'horsepower_3', 'horsepower_4', 'horsepower_5']], auto.mpg
kf = KFold(n_splits=10, shuffle=True, random_state=42)

In [39]:
#Loop for 5 degree polynomial linear regressions with k-Fold CV

kf_poly = {}

for i in range(1, 6):
    kf_mse = []
    
    for train_index, test_index in kf.split(x):
        x_train, x_test = x.iloc[train_index], x.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        if i == 1:
            X_TRAIN = x_train.iloc[:,0:1].values.reshape(-1, 1)
            X_TEST = x_test.iloc[:,0:1].values.reshape(-1, 1)
        else:
            X_TRAIN = x_train.iloc[:,0:i]
            X_TEST = x_test.iloc[:,0:i]
            
        kf_mse.append(
            mean_squared_error(
                y_test,
                LinearRegression(fit_intercept=True)
                .fit(
                    X_TRAIN,
                    y_train
                )
                .predict(
                    X_TEST
                )
            )
        )
        
        kf_poly['lm' + str(i) + '_MSE'] = np.array(kf_mse).mean()

In [40]:
kf_poly

{'lm1_MSE': 24.199808197692477,
 'lm2_MSE': 19.228636614268016,
 'lm3_MSE': 19.26626534663184,
 'lm4_MSE': 19.351092272804404,
 'lm5_MSE': 19.02323300932039}

In [54]:
portfolio = pd.read_csv('./Portfolio.csv', index_col=0)

In [55]:
def alpha_fn(data, start_index, end_index):
    X = data['X'][start_index:end_index]
    Y = data['Y'][start_index:end_index]
    return ((np.var(Y) - np.cov(X, Y)[0][1]) / (np.var(X) + np.var(Y) - 2*np.cov(X, Y)[0][1]))

In [56]:
alpha_fn(portfolio, 0, 100)

0.5766511516104116

In [59]:
from sklearn.utils import resample

portfolio_bs = resample(portfolio, replace = True, n_samples=100)
alpha_fn(portfolio_bs, 0, 100)

0.5668381936119579

In [61]:
#Bootstrap is removed from sklearn

bs_alpha = []
for i in range(0, 1000):
    bs_alpha.append(
        alpha_fn(resample(portfolio, replace=True, n_samples=100), 0, 100)
    )

bs_alpha = np.array(bs_alpha)

print('Bootstrapped alpha:', bs_alpha.mean())
print('SE:', bs_alpha.std())

Bootstrapped alpha: 0.5800971974821382
SE: 0.09105168734966321


In [62]:
def boot_fn(data, start_index, end_index):
    m = LinearRegression(fit_intercept=True).fit(
        data['horsepower'][start_index:end_index].values.reshape(-1, 1),
        data['mpg'][start_index:end_index]
    )
    
    return m.intercept_, m.coef_
boot_fn(auto, 0, 392)

(39.93586102117047, array([-0.15784473]))

In [68]:
boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)

(39.24400506607824, array([-0.15239138]))

In [71]:
bs_boot = {'t1':[], 't2':[]}

for i in range(0, 1000):
    bs_boot['t1'].append(
        boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[0]
    )
    bs_boot['t2'].append(
        boot_fn(resample(auto, replace=True, n_samples=392), 0, 392)[1][0]
    )

t1_es = np.array(bs_boot['t1']).mean()
t1_se = np.array(bs_boot['t1']).std()
t2_es = np.array(bs_boot['t2']).mean()
t2_se = np.array(bs_boot['t2']).std()

print('t1 be estimate & se:', t1_es, t1_se)
print('t2 be estimate & se:', t2_es, t2_se)

t1 be estimate & se: 39.97374191303202 0.8808238746469559
t2 be estimate & se: -0.15784878042991698 0.007369907972794244


In [81]:
import statsmodels.formula.api as sm

ols = sm.ols('mpg ~ horsepower', data=auto).fit()
ols.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,39.9359,0.717,55.660,0.000,38.525,41.347
horsepower,-0.1578,0.006,-24.489,0.000,-0.171,-0.145


In [76]:
def boot_fn2(data, start_index, end_index):
    m = LinearRegression(fit_intercept=True).fit(
        data[['horsepower', 'horsepower_2']][start_index:end_index],
        data['mpg'][start_index:end_index]
    )
    
    return m.intercept_, m.coef_

In [80]:
bs_boot2 = {'t1' : [], 't2' : [], 't3' : []}

for i in range(0, 1000):
    bs_boot2['t1'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[0]
    )
    bs_boot2['t2'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][0]
    )
    bs_boot2['t3'].append(
        boot_fn2(resample(auto, replace=True, n_samples=392), 0, 392)[1][1]
    )
    
t1_es = np.array(bs_boot2['t1']).mean()
t1_se = np.array(bs_boot2['t1']).std()
t2_es = np.array(bs_boot2['t2']).mean()
t2_se = np.array(bs_boot2['t2']).std()
t3_es = np.array(bs_boot2['t3']).mean()
t3_se = np.array(bs_boot2['t3']).std()

print('t1 be estimate & se:', t1_es, t1_se)
print('t2 be estimate & se:', t2_es, t2_se)
print('t3 be estimate & se:', t3_es, t3_se)

t1 be estimate & se: 56.94839505025298 2.081158689938737
t2 be estimate & se: -0.46440622184697006 0.033096363575907554
t3 be estimate & se: 0.0012324460668251985 0.00012167946010491052


In [78]:
import statsmodels.formula.api as sm

ols2 = sm.ols('mpg ~ horsepower + horsepower_2', data=auto).fit()
ols2.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,56.9001,1.800,31.604,0.000,53.360,60.440
horsepower,-0.4662,0.031,-14.978,0.000,-0.527,-0.405
horsepower_2,0.0012,0.000,10.080,0.000,0.001,0.001
